In [24]:
import pandas as pd

In [72]:
train = pd.read_csv("../data/liver-data-processed/train.csv")
dev = pd.read_csv("../data/liver-data-processed/devSet.csv")
test = pd.read_csv("../data/liver-data-processed/test.csv")

data = pd.concat([train, dev, test])

data.REGION = data.REGION.astype(int)
cron = data.groupby("REGION")["DAYSWAIT_CHRON"].mean().to_frame().reset_index()
cron.rename(columns = {"DAYSWAIT_CHRON": "WaitingTime"}, inplace = True)
print cron
data = data.loc[:, ("REGION", "GSTATUS", "GTIME")]
N = data.loc[data["GSTATUS"] == 1, :].shape[0]

    REGION  WaitingTime
0        1   257.038528
1        2   207.228608
2        3   121.850287
3        4   248.836022
4        5   299.653453
5        6   221.588235
6        7   210.181089
7        8   220.137522
8        9   257.451602
9       10   162.399387
10      11   171.378999


In [73]:
mask_3_m = (data["GTIME"] <= 90) & (data["GSTATUS"] == 1)
mask_6_m = (data["GTIME"] <= 6 * 30) & (data["GSTATUS"] == 1)
mask_1_y = (data["GTIME"] <= 365) & (data["GSTATUS"] == 1)
mask_3_y = (data["GTIME"] <= 3 * 365) & (data["GSTATUS"] == 1)

In [74]:
three_months = data.loc[mask_3_m, ["REGION", "GSTATUS"]].groupby("REGION").count().reset_index()
three_months["GSTATUS"] = 100 - (three_months["GSTATUS"] * 100.0 / N)
three_months.rename(columns = {"GSTATUS": "status_3_m"}, inplace = True)
three_months

,REGION,status_3_m
0,1,99.013897
1,2,96.718066
2,3,95.815724
3,4,98.035408
4,5,96.672378
5,6,99.447935
6,7,97.669903
7,8,98.435180
8,9,97.483343
9,10,97.650866


In [75]:
six_months = data.loc[mask_6_m, ["REGION", "GSTATUS"]].groupby("REGION").count().reset_index()
six_months["GSTATUS"] = 100 - (six_months["GSTATUS"] * 100.0 / N)
six_months.rename(columns = {"GSTATUS": "status_6_m"}, inplace = True)
six_months

,REGION,status_6_m
0,1,98.682658
1,2,95.526366
2,3,94.559299
3,4,97.395774
4,5,95.655816
5,6,99.189035
6,7,96.954121
7,8,97.959261
8,9,96.877974
9,10,97.034076


In [76]:
one_year = data.loc[mask_1_y, ["REGION", "GSTATUS"]].groupby("REGION").count().reset_index()
one_year["GSTATUS"] = 100 - (one_year["GSTATUS"] * 100.0 / N)
one_year.rename(columns = {"GSTATUS": "status_1_y"}, inplace = True)
one_year

,REGION,status_1_y
0,1,98.298115
1,2,93.881591
2,3,92.632781
3,4,96.565772
4,5,94.151913
5,6,98.979631
6,7,95.956596
7,8,97.262517
8,9,95.937559
9,10,96.025128


In [77]:
three_years = data.loc[mask_3_y, ["REGION", "GSTATUS"]].groupby("REGION").count().reset_index()
three_years["GSTATUS"] = 100 - (three_years["GSTATUS"] * 100.0 / N)
three_years.rename(columns = {"GSTATUS": "status_3_y"}, inplace = True)
three_years

,REGION,status_3_y
0,1,97.658481
1,2,90.748144
2,3,88.814011
3,4,94.445079
4,5,91.315439
5,6,98.419951
6,7,93.969160
7,8,95.880449
8,9,93.988197
9,10,93.999619


In [78]:
regions = pd.read_csv("../data/wage_gap_all.csv")
regions = regions.loc[:, ["Region", "State"]]
regions.columns = ["REGION", "STATE"]
print regions.loc[:10, :]

    REGION STATE
0        1    RI
1        1    MA
2        1    CT
3        1    ME
4        1    NH
5        2    DE
6        2    DC
7        2    MD
8        2    NJ
9        2    PA
10       2    WV


In [79]:
merged_data = pd.merge(three_months, regions, on="REGION")
merged_data = pd.merge(merged_data, six_months, on="REGION")
merged_data = pd.merge(merged_data, one_year, on="REGION")
merged_data = pd.merge(merged_data, three_years, on="REGION")
merged_data = pd.merge(merged_data, cron, on="REGION")
print merged_data

    REGION  status_3_m STATE  status_6_m  status_1_y  status_3_y  WaitingTime
0        1   99.013897    RI   98.682658   98.298115   97.658481   257.038528
1        1   99.013897    MA   98.682658   98.298115   97.658481   257.038528
2        1   99.013897    CT   98.682658   98.298115   97.658481   257.038528
3        1   99.013897    ME   98.682658   98.298115   97.658481   257.038528
4        1   99.013897    NH   98.682658   98.298115   97.658481   257.038528
5        1   99.013897    VT   98.682658   98.298115   97.658481   257.038528
6        2   96.718066    DE   95.526366   93.881591   90.748144   207.228608
7        2   96.718066    DC   95.526366   93.881591   90.748144   207.228608
8        2   96.718066    MD   95.526366   93.881591   90.748144   207.228608
9        2   96.718066    NJ   95.526366   93.881591   90.748144   207.228608
10       2   96.718066    PA   95.526366   93.881591   90.748144   207.228608
11       2   96.718066    WV   95.526366   93.881591   90.748144

In [80]:
merged_data = merged_data[["REGION", "STATE", "status_3_m", "status_6_m", "status_1_y", "status_3_y", "WaitingTime"]]
print merged_data

    REGION STATE  status_3_m  status_6_m  status_1_y  status_3_y  WaitingTime
0        1    RI   99.013897   98.682658   98.298115   97.658481   257.038528
1        1    MA   99.013897   98.682658   98.298115   97.658481   257.038528
2        1    CT   99.013897   98.682658   98.298115   97.658481   257.038528
3        1    ME   99.013897   98.682658   98.298115   97.658481   257.038528
4        1    NH   99.013897   98.682658   98.298115   97.658481   257.038528
5        1    VT   99.013897   98.682658   98.298115   97.658481   257.038528
6        2    DE   96.718066   95.526366   93.881591   90.748144   207.228608
7        2    DC   96.718066   95.526366   93.881591   90.748144   207.228608
8        2    MD   96.718066   95.526366   93.881591   90.748144   207.228608
9        2    NJ   96.718066   95.526366   93.881591   90.748144   207.228608
10       2    PA   96.718066   95.526366   93.881591   90.748144   207.228608
11       2    WV   96.718066   95.526366   93.881591   90.748144

In [81]:
merged_data.to_csv("../data/liver_status_new.csv", index=False)